In [1]:
import os
import time
import shutil
import pandas as pd
import win32com.client as client

from selenium import webdriver
from datetime import datetime
from sqlalchemy import create_engine, text
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support import expected_conditions as EC

start_time = time.time()

# Các function cần thiết

In [2]:
# Lấy ra dataframe từ sql query
def get_data_from_sql(query):
    server = 'VNSGNBIDC002'
    database = 'DR_Report'
    engine = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
    conn = engine.connect()
    sql_query = pd.read_sql_query(text(query), conn)
    df = pd.DataFrame(sql_query)
    conn.close()
    return df

# Run các câu lệnh trong SQL Query
def run_sql_query(query):
    server = 'VNSGNBIDC002'
    database = 'DR_Report'
    engine = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
    conn = engine.connect()
    conn.execute(text(query))
    conn.commit()
    conn.close()

# Remove các file đã tải ngày hôm nay và đã lưu ở folder chỉ định
def remove_all_file(start,end,filename):
    start_path = os.path.join(start,filename)
    end_path = os.path.join(end,filename)
    if os.path.isfile(start_path):
        os.remove(start_path)
        print("Đã xóa File Download")
    if os.path.isfile(end_path):
        os.remove(end_path)
        print("Đã xóa File Move")

# Move File mới (update) đến folder chỉ định
def move_file(start,end,filename):
    start_path = os.path.join(start,filename)
    end_path = os.path.join(end,filename)
    check = False
    while check == False:
        if os.path.isfile(start_path):
            shutil.copy2(start_path, end,follow_symlinks=True)
            os.remove(start_path)
            if os.path.isfile(end_path):
                check = True
                print("Download And Move File Success")
        else:
            print("Download And Move File Failture")

# Các thông tin cần input

In [3]:
current_datetime = datetime.now()
current_file =  current_datetime.strftime("data_ape_%d_%m_%Y.xls")
start = "C:/Users/HUNGU16/Downloads"
end = "C:/File_Downloaded_From_Ebao/APE1"
start_path = os.path.join(start,current_file)
end_path = os.path.join(end,current_file)

# Lấy Year Month dựa trên ngày Cutoff

In [4]:
query = '''
    DECLARE @CurrentDate DATE = GETDATE()
    SELECT*FROM DC_Source.STRUCTURE.Cut_Off
    WHERE startdate <= @CurrentDate AND enddate >= @CurrentDate
'''
df_ym = get_data_from_sql(query)
current_ym = df_ym['runnum'].values[0]

# DOWWNLOAD AND MOVE FILE

In [5]:
input_link = f"http://vnsgnebao001.chubblife.com.vn/life/servlet/com.acevn.report.general.APE_Report?sAction=DOWNLOAD_EXCEL&p_yearmonth={current_ym}&p_type_ape=1"
remove_all_file(start,end,current_file)
check_download = False
while check_download == False:
    options = webdriver.EdgeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    s = Service('C:\Selenium\msedgedriver.exe')
    driver = webdriver.Edge(service=s, options=options)
    driver.get(input_link)
    username_input = driver.find_element(by = 'name', value = 'userName')
    password_input = driver.find_element(by = 'name', value= "userPassword")
    username_input.send_keys("qulai0")
    password_input.send_keys("Hcm1234123456789@")
    login_button = driver.find_element(by="name", value="loginSubmitButton")
    login_button.click()
    time.sleep(20)
    if os.path.isfile(start_path):
        driver.quit()
        check_download = True
        move_file(start,end,current_file)
    else:
        driver.quit()
        time.sleep(40)

Download And Move File Success


# LOAD DATA TO SQL SERVER

In [6]:
# Tạo dataframe
df = pd.read_excel(end_path,skiprows=4)
current_date = current_datetime.strftime('%d-%m-%Y')
df['IMPORT_DATE'] = current_date

In [7]:
# Xóa các dòng dữ liệu cũ trước khi update
run_sql_query(f"DELETE FROM [DR_Report].[TMP].[Test_ETL_APE1] WHERE IMPORT_DATE = '{current_date}'")

In [8]:
# Data trước khi update
df_start = get_data_from_sql('SELECT*FROM [DR_Report].[TMP].[Test_ETL_APE1]')

In [9]:
# Sử dụng các column đang có trong sql cho dataframe
data = df[df_start.columns]

In [10]:
# Import các dòng dữ liệu mới vào SQL (Có vòng lặp nên phải làm tay)

server = 'VNSGNBIDC002'
database = 'DR_Report'
engine = create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
conn = engine.connect()
for index, row in data.iterrows():
    conn.execute(
        text(
            f'''
                INSERT INTO [DR_Report].[TMP].[Test_ETL_APE1](
                    [ITEM_ID],[POLICY_CODE],[PRODUCT_CODE],[PRODUCT_RATIO],[BASIC_PLAN_CODE],[POLICY_ISSUE_DATE],
                    [POLICY_EFF_DATE],[CVG_ISSUE_DATE],[CVG_EFF_DT],[POLICY_ORGAN_ID],[NB],[APE_NOT_TAKEN],
                    [APE_INCREASING_FA],[EXCESS_PREM],[TER_RIDER],[DECREASE_SA],[APE_TOTAL],[APE_EFFECTIVE_DATE],
                    [APE_AGENT_CODE],[APE_AGENT_ORGAN],[IMPORT_DATE]
                ) 
                VALUES(
                    {row['ITEM_ID']},{row['POLICY_CODE']},'{row['PRODUCT_CODE']}',{row['PRODUCT_RATIO']},'{row['BASIC_PLAN_CODE']}','{row['POLICY_ISSUE_DATE']}',
                    '{row['POLICY_EFF_DATE']}','{row['CVG_ISSUE_DATE']}','{row['CVG_EFF_DT']}','{row['POLICY_ORGAN_ID']}',{row['NB']},{row['APE_NOT_TAKEN']},
                    {row['APE_INCREASING_FA']},{row['EXCESS_PREM']},{row['TER_RIDER']},{row['DECREASE_SA']},{row['APE_TOTAL']},'{row['APE_EFFECTIVE_DATE']}',
                    '{row['APE_AGENT_CODE']}','{row['APE_AGENT_ORGAN']}','{row['IMPORT_DATE']}'  
                )
            '''
        )         
    )
    conn.commit()
conn.close()
end_time = time.time()
total_time = str(int(end_time - start_time)) + 's'

In [11]:
# Kiểm tra lại data đã import
df_end = get_data_from_sql('SELECT*FROM [DR_Report].[TMP].[Test_ETL_APE1]')
change_rows = len(df_end) - len(df_start)
text_check = f'Tổng thời gian ETL: {total_time}\nTổng số row được update {change_rows}' if change_rows > 0 else f'Không có data mới'
print(text_check)

Tổng thời gian ETL: 185s
Tổng số row được update 9491


# Gửi Email thông báo kết quả ETL

In [12]:
df_run = pd.DataFrame(
    {
        'Today': [len(df_end)],
        'Yesterday': [len(df_start)],
        'Change': [change_rows],
        'Run time': [total_time]
    }
).to_html(index = False)

style_table = '''
	<style>
		table {
			border:1px solid #b3adad;
			border-collapse:collapse;
			padding:5px;
		}
		table th {
			border:1px solid #b3adad;
			padding:5px;
			background: #00ff9d;
			color: #313030;
		}
		table td {
			border:1px solid #b3adad;
			text-align:center;
			padding:5px;
			background: #ffffff;
			color: #313030;
		}
	</style>
'''

In [13]:
outlook = client.Dispatch('Outlook.Application') 
olNS = outlook.GetNameSpace('MAPI')
email = outlook.CreateItem(0)
email.To = 'HuyHoang.Nguyen@Chubb.com;ThiTuongVy.Tran@chubb.com'
email.Subject = f'Import Data From EBAO (APE1)'
email.Body = f"Hi mn Iemm đã import data"
email.HTMLBody = f'''
    <html><body>
        {style_table}
        {df_run}
    <html><body>
'''
email.Send()